## Data Lake S3 Local 
- Do not use with large datasets

In [1]:
from pyspark.sql import SparkSession
import os
import configparser

In [2]:
# LOAD AWS CREDENTIALS AS ENV VARS
config = configparser.ConfigParser()
config.read_file(open('/Users/brad/.aws/credentials'))

aws_access_key_id = config.get('default','aws_access_key_id')
aws_secret_access_key = config.get('default','aws_secret_access_key')

# DID NOT WORK LIKE IN EXAMPLE
# os.environ["AWS_ACCESS_KEY_ID"]= config['default']['aws_access_key_id']
# os.environ["AWS_SECRET_ACCESS_KEY"]= config['default']['aws_secret_access_key']

In [1]:
# CREATE SPARK SESSION WITH HADOOP-AWS PACKAGE
spark = SparkSession.builder\
    .config('spark.jars.packages', "org.apache.hadoop:hadoop-aws:2.7.0")\
    .getOrCreate()

NameError: name 'SparkSession' is not defined

In [4]:
sc=spark.sparkContext
hadoop_conf=sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3.awsAccessKeyId", aws_access_key_id)
hadoop_conf.set("fs.s3.awsSecretAccessKey", aws_secret_access_key)

In [5]:
# LOAD DATA FROM S3
df = spark.read.csv("s3n://brad-data-01/demo-data/payments.csv")

Py4JJavaError: An error occurred while calling o31.csv.
: java.io.IOException: No FileSystem for scheme: s3n
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:376)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:297)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:286)
	at scala.Option.getOrElse(Option.scala:189)
	at 

In [21]:
df.printSchema()
df.show(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|       _c0|        _c1|     _c2|      _c3|   _c4|                 _c5|
+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



In [22]:
# Infer schema, fix header and seperator
df = spark.read.csv("s3n://brad-data-01/demo-data/payments.csv", sep=",", inferSchema=True, header=True)

In [19]:
df.printSchema()
df.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: string (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



In [14]:
# FIX IT DATE TYPE
import pyspark.sql.functions as f
dfPayment = df.withColumn("payment_date", f.to_timestamp("payment_date"))
dfPayment.printSchema()
dfPayment.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: timestamp (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 23:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 17:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 23:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 02:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 10:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



In [15]:
# EXTRACT THE MONTH
dfPayment = dfPayment.withColumn("month", f.month("payment_date"))
dfPayment.show(5)

+----------+-----------+--------+---------+------+--------------------+-----+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|month|
+----------+-----------+--------+---------+------+--------------------+-----+
|     16050|        269|       2|        7|  1.99|2017-01-24 23:40:...|    1|
|     16051|        269|       1|       98|  0.99|2017-01-25 17:16:...|    1|
|     16052|        269|       2|      678|  6.99|2017-01-28 23:44:...|    1|
|     16053|        269|       2|      703|  0.99|2017-01-29 02:58:...|    1|
|     16054|        269|       1|      750|  4.99|2017-01-29 10:10:...|    1|
+----------+-----------+--------+---------+------+--------------------+-----+
only showing top 5 rows



In [16]:
# COMPUTER AGGREGATE REVENUE PER MONTH
dfPayment.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month, sum(amount) as revenue
    FROM payment
    GROUP BY month
    ORDER BY revenue desc
""").show()

+-----+------------------+
|month|           revenue|
+-----+------------------+
|    4|26881.290000003297|
|    3|23886.560000002115|
|    2| 9631.879999999608|
|    1| 4692.739999999863|
|    5|  514.180000000001|
+-----+------------------+



In [41]:
# FIX THE SCHEMA
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, TimestampType as Date
paymentSchema = R([
    Fld("payment_id", Int()),
    Fld("customer_id", Int()),
    Fld("staff_id", Int()),
    Fld("rental_id", Int()),
    Fld("amount", Dbl()),
    Fld("payment_date", Date()),
])

In [42]:
dfPaymentWithSchema = spark.read.csv("s3n://brad-data-01/demo-data/payments.csv", sep=",", schema=paymentSchema, header=True)

In [43]:
dfPaymentWithSchema.printSchema()
df.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: timestamp (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



In [45]:
dfPaymentWithSchema.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month(payment_date) as m, round(sum(amount), 2) as revenue
    FROM payment
    GROUP BY m
    ORDER BY revenue desc
""").show()

+---+--------+
|  m| revenue|
+---+--------+
|  4|26881.29|
|  3|23886.56|
|  2| 9631.88|
|  1| 4692.74|
|  5|  514.18|
+---+--------+

